## Signal Processing
The plots created in the last section show a strong annual harmonic component. Lets, identify this components, subtract them from the signal and the re-plot to see if there is any transannual trend.


In [ ]:
import pandas as pd
import numpy as np
from funciones import*
import matplotlib.pyplot as plt
import funciones


from importlib import reload

Data=pd.read_csv("../../data/Time_series_DATA.txt",skiprows=149,index_col=0,sep='\t')
Data.index = pd.to_datetime(Data.index, format="%Y-%m-%d")
explore_contents(Data,opt={'Sparsity':True,'Info':True,'Time History': False})
#Data=Data.drop(columns=['Days since start of year','Days until break up','Noisy predicted ice thickness [m]'])
#Data=pd.read_csv('https://raw.githubusercontent.com/iceclassic/sandbox/main/Data/Time_series_DATA.txt',index_col=0,skiprows=149)






## Stationary Time Series

In [ ]:
# Lets compute the mean

# de trend de data


# plot Spectrogram

# plot Autocorrelation formula

 

### Power Spectral density
A simple way to estimate the Power Spectral Density (PSD) of signal is using the [Welch method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html#id1).
> do we explain the theory behind the method?

:::{card} Exercise 1

Read the documentation for `compute_and_plot_psd()` function from the ice_classic package and estimate te PSD for all columns
¿ Do all the variable experience the same seasonal behavior? Is there any multiyear harmonic?

:::


In [ ]:
_=compute_and_plot_psd(Data,plot_period=False)	

# given the frequency of our signal, using the period (T) instead of the frequency (1/F) might be better 
PSD_info=compute_and_plot_psd(Data,plot_period=True,find_peaks_kwargs={"distance":100,"threshold":10})	

In the last figures, multiple peaks can be observed with the most prominent being the associated with a period of  350-370, which is  expected as is represent the yearly variation.

One common way to identify the peak of PSD is to use [`find_peaks()`](https://docs.scipy.org/doc/scipy//reference/generated/scipy.signal.find_peaks.html) from  `scipy`.The function `compute_and_plot_psd()` uses internally this function to find the peaks of each column, and saves them in a dictionary. 


::{card} Exercise 2
Extract the peak of each column and compare them.
```{admonition} Tips/Help
:class: tip, dropdown
Look-up the documentation of `[find_peaks]`(https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html). Considering what you expect the peaks to look like, change the values of `prominence`, `distance` and or `threshold` to refine the search for peaks
```
:::



In [ ]:
for col, psd_info in PSD_info.items():
    peak_periods =np.round(psd_info['peak_periods']).astype(int) 
    
    print(f"Column: {col}")
    print(f"Peak Periods: {peak_periods}\n")


import pprint
pprint.pprint(PSD_info)

Another issue wiht the plot above is that we can observe a lot of high-frequency component, which are most likely associate with noise.
Lets apply a low-pass filter, that block higher frequencies and lets *low frequencies pass* and re-plot.

> show that rolling mean can be thougth of as a low pass filter?

:::{card} Exercise 4
Use the same function as before, but consider the argument chagninf the default of  `apply_filter`, `max_allowed_freq`and `filter_order`
:::




In [ ]:
_=compute_and_plot_psd(Data,plot_period=False,find_peaks_kwargs={"distance":100,"threshold":10,"height":1e-2},apply_filter=True,max_allowed_freq=1/14,filter_order=4)
PSD_info=compute_and_plot_psd(Data,plot_period=True,find_peaks_kwargs={"distance":100,"threshold":10,"height":1e-2},apply_filter=True,max_allowed_freq=1/30,filter_order=4)

In [ ]:
for col, psd_info in PSD_info.items():
    peak_periods =np.round(psd_info['peak_periods']).astype(int) 
    
    print(f"Column: {col}")
    print(f"Peak Periods: {peak_periods}\n")

> put here plot of historic seasonality+ harmonic identified


In [ ]:
def generate_harmonics_with_amplitude(index, peak_periods, peak_psd_values, num_harmonics=1):
    """Generate harmonic components based on peak periods and their amplitudes."""
    harmonics = np.zeros(len(index))
    
    for period, amplitude in zip(peak_periods, peak_psd_values):
        frequency = 2 * np.pi / period
        for n in range(1, num_harmonics + 1):
            harmonics += amplitude * np.sin(n * frequency * np.arange(len(index)))
    
    return harmonics
def compute_residuals(df, psd_info, num_harmonics=1):
    """Compute residuals by subtracting harmonic components from data using peak information."""
    residuals_df = pd.DataFrame(index=df.index)
    
    for col, info in psd_info.items():
        if col in df.columns:
            peak_periods = np.round(info['peak_periods']).astype(int)
            peak_psd_values = info['peak_psd_values']
            
            harmonics = generate_harmonics_with_amplitude(df.index, peak_periods, peak_psd_values, num_harmonics)
            residuals_df[col] = df[col] - harmonics

    return residuals_df


In [ ]:


# Compute residuals
residuals_df = compute_residuals(Data, psd_info, num_harmonics=3)
explore_contents(residuals_df)